In [1]:
import cv2 as cv
import numpy as np
from glob import glob
from sklearn.utils import shuffle
import time
import pandas as pd
import pickle
from config import efficientNet_config
from models import efficientNetV2B0_model, efficientNetV2B3_model
from utils import plt_heatmap, plt_roccurve

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

In [2]:
model = efficientNetV2B0_model()
model.load_weights('../model/202211/20221102.weights')

2023-03-07 12:37:13.788441: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-07 12:37:13.788887: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-07 12:37:13.794211: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-07 12:37:13.794934: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-07 12:37:13.795205: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b0 (Function  (None, 1280)             5919312   
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 5,920,593
Trainable params: 5,859,985
Non-trainable params: 60,608
_________________________________________________________________


In [3]:
height  = efficientNet_config['height_B0']
width   = efficientNet_config['width_B0']

In [13]:
testdata = "test_q"
test_dir      = '../dataset_test_everyframes/{}/'.format(testdata)
## everyframes
img_paths = sorted(glob(test_dir + "*.png"))

start_time = time.time()
s_time     = time.time()
loop_times = []

pred_results = []
img_names = []
for i,img_path in enumerate(img_paths):
    # imege
    img_name  = img_path.split("/")[-1]
    img_array = cv.imread(img_path)
    img_array = cv.resize(img_array,(height,width))     # resize to (224,224)
    
    
    pred_result = model.predict(np.array([img_array]))
    pred_result = np.where(pred_result > 0.5, 1, 0)
    
    if pred_result[0][0]!=0:
        print(pred_result[0][0])
        pred_results.append(pred_result[0][0])
        img_names.append(img_name)

    # 計算每張圖片計算時間
    temp_time = time.time()
    loop_times.append(temp_time-s_time)
    s_time = temp_time

end_time = time.time()

print(end_time-start_time)

1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 19ms/step
1
1/1 [==============================] - 0s 18ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 18ms/step
1
1/1 [==============================] - 0s 21ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [=======================

In [14]:
print("min time on stage 3=",min(loop_times))
print("Max time on stage 3=",max(loop_times))
print("Avg time on stage 3=",sum(loop_times)/len(loop_times))

min time on stage 3= 0.04057884216308594
Max time on stage 3= 0.18789911270141602
Avg time on stage 3= 0.04494517366687728


In [ ]:
stage1_everyframes_result = pd.DataFrame({"stage1_pred_result":pred_results,"img_name":img_names})

In [ ]:
# with open('../dataset_test_everyframes/{}_stage1_everyframes_result_20230110.pk'.format(testdata), 'wb') as f:
#     pickle.dump(stage1_everyframes_result, f)